In [1]:
from qiskit import *
from qiskit.visualization import plot_histogram

from random import getrandbits, randint

from binarycostfunction import *
from customgates import *

In [36]:
test_bcf = BinaryCostFunction(4)

clauses = [
    BinaryClause("1XXX"),
    BinaryClause("XXX1")
]

values = [1, 2]

for clause, value in zip(clauses, values):
    test_bcf.add_clause(clause, value)
    
print(test_bcf)

bcf1 = BinaryCostFunction(4)

clauses = [
    BinaryClause("X1X1"),
    BinaryClause("XXX0"),
    BinaryClause("X011"),
    BinaryClause("XXX1"),
    BinaryClause("01X1")
]

values = [6, 3, 7, -2, -2]

for clause, value in zip(clauses, values):
    bcf1.add_clause(clause, value)
    
print(bcf1)

1 * 1XXX
2 * XXX1

6 * X1X1
3 * XXX0
7 * X011
-2 * XXX1
-2 * 01X1



In [37]:
def dec_to_bin(k, n):
    bin_str = ""
    for i in range(n):
        bin_str = str(k%2) + bin_str
        k = k//2
    return bin_str

def bin_to_dec_1(b):
    dec = 0
    for bi in b:
        dec *= 2
        dec += 1 if bi == "1" else 0
    return dec

def bin_to_dec_2(b):
    dec = 0
    l = 0
    for bi in b:
        dec *= 2
        dec += 1 if bi == "1" else 0
        l += 1
    if dec >= 2**(l-1):
        dec -= 2**l
    return dec

Circuit parameters:
  - `n`: Number of qubits for the first regiter, and number of variables of the polynomial cost function.
  - `m`: Number of qubits for the second register.
  - `bcf`: Binary cost function to optimize

In [38]:
statevector_sim = Aer.get_backend("statevector_simulator")

In [45]:
n = 4
m = 6
bcf = bcf1
backend = statevector_sim

#------

xs = [dec_to_bin(getrandbits(n), n)]
ys = [bcf.evaluate(xs[-1])]
rs = []

print("Initial conditions: x={0:s}, y={1:4d}".format(xs[-1], ys[-1]))
#------

k = 1
i = 1
while i <= 30:
    
    print("Iteration {0:4d}:".format(i), end = "")
    
    #Select number of rotations randomly
    rs.append(k)
    
    print(" r={0:2d}".format(rs[-1]), end = "")
    
    #Create and run quantum circuit
    qc = create_GAS_circuit(m, bcf, ys[-1], rs[-1])
    job_sim = execute(qc, backend, shots = 1)
    result_sim = job_sim.result()
    
    #Retrieve read x and y
    counts = result_sim.get_counts(qc)
    result = list(counts.keys())[0]
    y = result[0:m]
    y_val = bin_to_dec_2(result[0:m]) - ys[-1]
    x = result[m+1:m+1+n]
    
    print(" x={0:s}, y={1:s} y_val={2:4d}".format(x, y, y_val), end = "")
    
    #If y is better (smaller) than ys[-1]: Update threshold and reset k
    #If not: Do not update threshold, increase k
    if y_val < ys[-1]:
        xs.append(x)
        ys.append(y_val)
        k = 1
        print(" !! ", end = "")
    else:
        xs.append(xs[-1])
        ys.append(ys[-1])
        k = k + 1
        print("    ", end = "")
    
    print(" bext_x={0:s}, best_y={1:4d}".format(xs[-1], ys[-1]))
        
    #Update number of iterations
    i = i + 1    

Initial conditions: x=0011, y=   5
Iteration    1: r= 1 x=0111, y=000111 y_val=   2 !!  bext_x=0111, best_y=   2
Iteration    2: r= 1 x=0101, y=000100 y_val=   2     bext_x=0111, best_y=   2
Iteration    3: r= 2 x=1111, y=000110 y_val=   4     bext_x=0111, best_y=   2
Iteration    4: r= 3 x=0100, y=000101 y_val=   3     bext_x=0111, best_y=   2
Iteration    5: r= 4 x=1100, y=000101 y_val=   3     bext_x=0111, best_y=   2
Iteration    6: r= 5 x=0110, y=000101 y_val=   3     bext_x=0111, best_y=   2
Iteration    7: r= 6 x=1001, y=000000 y_val=  -2 !!  bext_x=1001, best_y=  -2
Iteration    8: r= 1 x=1001, y=111100 y_val=  -2     bext_x=1001, best_y=  -2
Iteration    9: r= 2 x=0001, y=111100 y_val=  -2     bext_x=1001, best_y=  -2
Iteration   10: r= 3 x=1000, y=000001 y_val=   3     bext_x=1001, best_y=  -2
Iteration   11: r= 4 x=1110, y=000001 y_val=   3     bext_x=1001, best_y=  -2
Iteration   12: r= 5 x=1011, y=000011 y_val=   5     bext_x=1001, best_y=  -2
Iteration   13: r= 6 x=1001, 

In [49]:
print(xs)
print(ys)

['0011', '0111', '0111', '0111', '0111', '0111', '0111', '1001', '1001', '1001', '1001', '1001', '1001', '1001', '1001', '1001', '1001', '1001', '1001', '1001', '1001', '1001', '1001', '1001', '1001', '1001', '1001', '1001', '1001', '1001', '1001']
[5, 2, 2, 2, 2, 2, 2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2]
